# Used to plot PMT gain against PMT voltage for the orange slide power changing experiment

http://labrigger.com/blog/2016/08/11/checking-pmt-performance-over-time/comment-page-1/

http://labrigger.com/blog/2010/07/30/measuring-the-gain-of-your-imaging-system/

In [1]:
import matplotlib.pyplot as plt
import os
import fnmatch
import numpy as np
from xml.dom.minidom import parse
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
def return_tStack(directory):
    """
    take a directory and return a tStack numpy array of all tiffs in there in shape (num_tifs,width,height)
    """
    
    os.chdir(directory)
    # construct list of all files in given directory
    list_of_files = os.listdir('.')
    
    # reduce the above list to list of files matching given pattern
    pattern = '*.tif'
    list_of_tifs = []
    for entry in list_of_files:
        if fnmatch.fnmatch(entry, pattern):
            list_of_tifs.append(entry)

    # get metadata of first gif, assuming all other are equal
    image = plt.imread(list_of_tifs[0])
    width = image.shape[0]
    height = image.shape[1]
    num_tifs = len(list_of_tifs)

    # construct empty tStack to be filled later
    tStack = np.zeros((num_tifs,width,height))

    # fill tStack with tifs
    for i in range(num_tifs):
        tStack[i] =  plt.imread(list_of_tifs[i])

    return tStack

In [3]:
def return_subfolder_and_xml(directory):
    """
    takes a directory, finds all subfolders of that directory and the .xml file describing them
    returns a dictionary with {subfolder:xml_filename}
    """
    os.chdir(directory)
    # construct list of all files in given directory

    # construct list of all subfolders in given directory
    list_of_subfolders = [f.path for f in os.scandir('.') if f.is_dir()]
    
    subfolders_xml_dict = {}

    # construct list of xmls for every subfolder
    for subfolder in list_of_subfolders:
        # chdir to directory/subfolder
        os.chdir(subfolder)
        
        # define pattern that will be matched
        pattern = '*.xml'
        
        # get list of files in current folder
        list_of_files = os.listdir('.')
        
        # construct list of all xmls in current folder
        list_of_xmls = []
        for f in list_of_files:
            if fnmatch.fnmatch(f, pattern):
                list_of_xmls.append(f)
        
        # add subfolder:xml to subfolders_xml_dict
        subfolders_xml_dict[subfolder] = list_of_xmls[0]
        os.chdir('..')

    # return first xml (we assume there is only one) and subfolder
    return subfolders_xml_dict

In [4]:
def return_power_states(xml,laser_name,pmt_number):
    """
    whatever
    """
    # parse xml file into minidom structure
    dom1 = parse(xml)
    
    # take first PVStateShard because we assume the powers not to change during       the tStack
    first_shard = dom1.getElementsByTagName('PVStateShard')[0]
    # produce list of every PVStateValue
    state_value_list = first_shard.getElementsByTagName('PVStateValue')
    
    # search thois lists keys for 'laserPower' and 'pmtGain'
    for i in range(len(state_value_list)):
        if state_value_list[i].getAttribute('key') == 'laserPower':
            laserPower_index = i
        if state_value_list[i].getAttribute('key') == 'pmtGain':
            pmtGain_index = i
    
    # create list of indexed values in 'laserPower' element
    indexed_value_list = state_value_list[laserPower_index].getElementsByTagName('IndexedValue')

    # search this list for 'laser_name' matching the input to this function
    for i in range(len(indexed_value_list)):
        if indexed_value_list[i].getAttribute('description') == laser_name:
            laser_power = indexed_value_list[i].getAttribute('value')
            
    # create list of indexed values in 'pmtGain' element
    indexed_value_list = state_value_list[pmtGain_index].getElementsByTagName('IndexedValue')

    # create pmt_str to fit the style in the xml
    pmt_str = 'PMT ' + pmt_number + ' HV'

    # search this list for 'laser_name' matching the input to this function
    for i in range(len(indexed_value_list)):
        if indexed_value_list[i].getAttribute('description') == pmt_str:
            pmt_gain = indexed_value_list[i].getAttribute('value')

    return (laser_power,pmt_gain)

In [5]:
#### set the things you are interested in here #####

directory = r'F:\Data\mloidolt\2018-08-02\2018-08-02_XscanPMT'
print(directory)
laser_name = 'Imaging'
pmt_number = '2'
t_stack_len = 100
width = 512
height = 512

F:\Data\mloidolt\2018-08-02\2018-08-02_XscanPMT


In [6]:
#### actual program starts ####

# get all of the subfolders with xml file descriptions
subfolders_xml_dict = return_subfolder_and_xml(directory)
print(subfolders_xml_dict)

# create a value_dict to be filled as {(laser_power,pmt_gain):(mean,variance)} later
value_dict = {}

# iterate through {(subfolder:xml file)} dictionary
for subfolder in subfolders_xml_dict.keys():
    print(subfolder)
    # read out (laser_power,pmt_gain) of xml file
    (laser_power,pmt_gain) = return_power_states(subfolder+'/'+subfolders_xml_dict[subfolder],laser_name,pmt_number)
    # get tStack of the subfolder as numpy array
    tStack = return_tStack(subfolder)
    # feed mean, variance of the whole tStack (along all axes) into dictionary
    value_dict[(laser_power,pmt_gain)] = (np.mean(tStack),np.var(tStack))
    os.chdir(directory)

np.save('mean_var_dict.npy', value_dict) 

{'.\\2018-08-02V200X0-009': '2018-08-02V200X0-009.xml', '.\\2018-08-02V200X0-010': '2018-08-02V200X0-010.xml', '.\\2018-08-02V200X0-011': '2018-08-02V200X0-011.xml', '.\\2018-08-02V200X100-012': '2018-08-02V200X100-012.xml', '.\\2018-08-02V200X1000-021': '2018-08-02V200X1000-021.xml', '.\\2018-08-02V200X1100-022': '2018-08-02V200X1100-022.xml', '.\\2018-08-02V200X1200-023': '2018-08-02V200X1200-023.xml', '.\\2018-08-02V200X1300-024': '2018-08-02V200X1300-024.xml', '.\\2018-08-02V200X1400-025': '2018-08-02V200X1400-025.xml', '.\\2018-08-02V200X1500-026': '2018-08-02V200X1500-026.xml', '.\\2018-08-02V200X1600-027': '2018-08-02V200X1600-027.xml', '.\\2018-08-02V200X1700-028': '2018-08-02V200X1700-028.xml', '.\\2018-08-02V200X1800-029': '2018-08-02V200X1800-029.xml', '.\\2018-08-02V200X1900-030': '2018-08-02V200X1900-030.xml', '.\\2018-08-02V200X200-013': '2018-08-02V200X200-013.xml', '.\\2018-08-02V200X2000-031': '2018-08-02V200X2000-031.xml', '.\\2018-08-02V200X300-014': '2018-08-02V200X

.\2018-08-02V200X0-010
.\2018-08-02V200X0-011
.\2018-08-02V200X100-012
.\2018-08-02V200X1000-021
.\2018-08-02V200X1100-022
.\2018-08-02V200X1200-023
.\2018-08-02V200X1300-024
.\2018-08-02V200X1400-025
.\2018-08-02V200X1500-026
.\2018-08-02V200X1600-027
.\2018-08-02V200X1700-028
.\2018-08-02V200X1800-029


KeyboardInterrupt: 

In [ ]:
# set modalities of the 2d-parameter-space scan
# do this manually!!

Pmins = [200,200,200]
Psteps = [50]*8

# PV that corresponds to 50mw OnSample
# maybe not
Pmaxs = [700,700,700]


Vmins = [345,445,495,545,595,645,695,745]
Vsteps = [5]*8

# Voltage that slightly oversaturates the uniformly lit sample
Vmaxs = [355,455,505,555,605,655,705,755]

# construct linspaces
c = len(Pmins)

Pspaces = []
for i in range(c):
    Pspace = np.arange(Pmins[i],Pmaxs[i]+Psteps[i],Psteps[i])
    print(Pspace)
    Pspaces.append(Pspace)

Vspaces = []
for i in range(c):
    Vspace = np.arange(Vmins[i],Vmaxs[i]+Vsteps[i],Vsteps[i])
    Vspaces.append(Vspace)

print(Pspaces)
print(Vspaces)

gains = []
offsets = []
std_errs = []

for i in range(c):
    Vlen = len(Vspaces[i])
    for j in range(Vlen):
        Plen = len(Pspaces[i])
        means = np.zeros(Plen)
        variances = np.zeros(Plen)
        for k in range(Plen):
            (means[k],variances[k]) = value_dict[str(Pspaces[i][k]),str(Vspaces[i][j])]
        slope, intercept, r_value, p_value, std_err = stats.linregress(means, variances)
        print(slope, intercept, r_value, p_value, std_err)
        gains.append(slope)
        offsets.append(intercept)
        std_errs.append(std_err)

plt.errorbar(np.asarray(Vspaces).flatten(),gains,yerr=std_errs,
             label='PMT 2, 765nm reflection',linestyle='None',marker='*')
plt.legend()
plt.xlabel('PMT Voltage')
plt.ylabel('PMT Gain')
plt.xlim(200,700)
#plt.ylim(0,500)
plt.show()

        

In [16]:
xdata = np.asarray(Vspaces).flatten()
ydata = gains
yerr = std_errs

exponential = lambda t,a,b: a * np.exp(b*t)

popt, pcov = optimize.curve_fit(exponential,  xdata,  ydata, sigma=yerr,  p0=(0.01,0.01))
print(popt,pcov)

tspace = np.arange(300,500,1)

plt.plot(tspace,exponential(tspace,*popt),label='exp fit')
plt.errorbar(np.asarray(Vspaces).flatten(),gains,yerr=std_errs,
             label='PMT 2, 765nm reflection',linestyle='None',marker='*')
plt.legend()
plt.show()

NameError: name 'optimize' is not defined